# Sartorius Segmentation - Detectron2 [diagnosis]

### Hi kagglers, This is `Training` notebook using `Detectron2`.
[Sartorius Segmentation - Detectron2 [training]](https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-detectron2-training) 
### Please if this kernel is useful, <font color='red'>please upvote !!</font>

## Other notebooks in this competition 
- [Sartorius Segmentation - Keras U-Net[Training]](https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-keras-u-net-training)
- [Sartorius Segmentation - Keras U-Net[Inference]](https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-keras-u-net-inference/edit)

# Detectron2 
Detectron2 is Facebook AI Research's next generation software system that implements state-of-the-art object detection algorithms. It is a ground-up rewrite of the previous version, Detectron, and it originates from maskrcnn-benchmark

## Install Detectron2 offline

In [ ]:
!pip install ../input/detectron-05/whls/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/detectron2-0.5/detectron2 --no-index --find-links ../input/detectron-05/whls

# importing libraries

In [ ]:
import detectron2
import torch
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from PIL import Image
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fastcore.all import *
import json
detectron2.__version__

# Functions

In [ ]:
# From https://www.kaggle.com/stainsby/fast-tested-rle
def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def get_masks(pred, shape, thres, mixp):
    take = pred['instances'].scores >= thres
    pred_masks = pred['instances'].pred_masks[take]
    pred_masks = pred_masks.cpu().numpy()
    res = []
    used = np.zeros(shape[:2], dtype=int)
    for mask in pred_masks:
        mask = mask * (1-used)
        if mask.sum() >= mixp: # skip predictions with small area
            used += mask
            res.append(mask)
    return res

import os
import skimage
import numpy as np
import pandas as pd
import skimage.segmentation
import matplotlib.pyplot as plt

def rles_to_mask(encs, shape):
    """
    Decodes a rle.

    Args:
        encs (list of str): Rles for each class.
        shape (tuple [2]): Mask size.

    Returns:
        np array [shape]: Mask.
    """
    img = np.zeros(shape[0] * shape[1], dtype=np.uint)
    for m, enc in enumerate(encs):
        if isinstance(enc, np.float) and np.isnan(enc):
            continue
        enc_split = enc.split()
        for i in range(len(enc_split) // 2):
            start = int(enc_split[2 * i]) - 1
            length = int(enc_split[2 * i + 1])
            img[start: start + length] = 1 + m
    return img.reshape(shape)

def precision_at(threshold, iou):
    """
    Computes the precision at a given threshold.

    Args:
        threshold (float): Threshold.
        iou (np array [n_truths x n_preds]): IoU matrix.

    Returns:
        int: Number of true positives,
        int: Number of false positives,
        int: Number of false negatives.
    """
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) >= 1  # Correct objects
    false_negatives = np.sum(matches, axis=1) == 0  # Missed objects
    false_positives = np.sum(matches, axis=0) == 0  # Extra objects
    tp, fp, fn = (
        np.sum(true_positives),
        np.sum(false_positives),
        np.sum(false_negatives),
    )
    return tp, fp, fn

def compute_iou(labels, y_pred):
    """
    Computes the IoU for instance labels and predictions.

    Args:
        labels (np array): Labels.
        y_pred (np array): predictions

    Returns:
        np array: IoU matrix, of size true_objects x pred_objects.
    """

    true_objects = len(np.unique(labels))
    pred_objects = len(np.unique(y_pred))

    #print()
    # Compute intersection between all objects
    try:
        intersection = np.histogram2d(
            labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects)
        )[0]

        # Compute areas (needed for finding the union between all objects)
        area_true = np.histogram(labels, bins=true_objects)[0]
        area_pred = np.histogram(y_pred, bins=pred_objects)[0]
        area_true = np.expand_dims(area_true, -1)
        area_pred = np.expand_dims(area_pred, 0)

        # Compute union
        union = area_true + area_pred - intersection
        iou = intersection / union

        return iou[1:, 1:]  # exclude background
    except:
        return np.zeros([true_objects-1, pred_objects-1])

def iou_map(truths, preds, verbose=0):
    """
    Computes the metric for the competition.
    Masks contain the segmented pixels where each object has one value associated,
    and 0 is the background.

    Args:
        truths (list of masks): Ground truths.
        preds (list of masks): Predictions.
        verbose (int, optional): Whether to print infos. Defaults to 0.

    Returns:
        float: mAP.
    """
    ious = [compute_iou(truth, pred) for truth, pred in zip(truths, preds)] 
    #print(ious[0].shape)

    if verbose:
        print("Thresh\tTP\tFP\tFN\tPrec.")

    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tps, fps, fns = 0, 0, 0
        for iou in ious:
            tp, fp, fn = precision_at(t, iou)
            tps += tp
            fps += fp
            fns += fn

        p = tps / (tps + fps + fns)
        prec.append(p)

        if verbose:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tps, fps, fns, p))

    if verbose:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))

    return np.mean(prec)

def get_all_iou(preds, true_masks, thres, mixp):
    pred_masks = []
    for pred in preds:
        _pred_masks = get_masks(pred, true_masks[0].shape, thres, mixp)
        pred_masks.append(sum([_pred_masks[i]*(i+1) for i in range(len(_pred_masks))]))
    #cv2.imwrite('pred_masks.png', pred_masks[0]*255)
    #cv2.imwrite('true_masks.png', true_masks[0]*255)
    #a =1/0
    return iou_map(true_masks, pred_masks)

def get_bad_preds(preds, true_masks, thres, mixp, n_preds=10):
    bad_pred_dict_list = []
    for pred, true_mask in zip(preds, true_masks):
        _pred_mask = get_masks(pred, true_masks[0].shape, thres, mixp)
        pred_mask = sum([_pred_mask[i]*(i+1) for i in range(len(_pred_mask))])
        score = iou_map([true_mask], [pred_mask])
        if len(bad_pred_dict_list) == 0 or bad_pred_dict_list[0]['score']>score:
            if len(bad_pred_dict_list) == n_preds:
                del bad_pred_dict_list[0]
            bad_pred_dict_list.append({'pred_mask': pred_mask, 'true_mask': true_mask, 'score': score, 
                                       'image_path': pred['image_path']})
            bad_pred_dict_list.sort(key=lambda d: d['score'], reverse=True)
        
    return bad_pred_dict_list

def do_eval(checkpoint):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(MODEL_NAME))
    cfg.INPUT.MASK_FORMAT='bitmask'
    cfg.MODEL.WEIGHTS = checkpoint 
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
    #cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.40
    cfg.TEST.DETECTIONS_PER_IMAGE = 1000
    try:
        predictor = DefaultPredictor(cfg)
    except:
        cfg.MODEL.DEVICE = "cpu"
        predictor = DefaultPredictor(cfg)
        
    pred_list_by_class = [[], [], []]
    ture_masks_by_class = [[], [], []]
    for image in valid_images:
        fn = str(Dir_testdata) + '/' + image['file_name']
        im = cv2.imread(fn)
        pred = predictor(im)
        pred['image_path'] = fn
        pred_class = torch.mode(pred['instances'].pred_classes)[0]
        pred_list_by_class[pred_class].append(pred)

        fid = image['id']
        df = train_df[train_df['id']==fid]
        shape = df[['height', 'width']].values[0]
        rles = df['annotation'].values[0]
        masks = rles_to_mask(rles, shape).astype(np.uint16)
        ture_masks_by_class[pred_class].append(masks)
        
    iou = 0.0
    n_all_preds = 0
    for cls_i in range(len(pred_list_by_class)):
        preds = pred_list_by_class[cls_i]
        ture_masks = ture_masks_by_class[cls_i]
        ious = get_all_iou(preds, ture_masks, THRESHOLDS[cls_i], MIN_PIXELS[cls_i])
        iou += np.mean(ious) * len(preds)
        n_all_preds += len(preds)
    iou /= n_all_preds
    
    return iou

In [ ]:
Dir_testdata=Path('../input/sartorius-cell-instance-segmentation')
ids, masks=[],[]
test_image_names = (Dir_testdata/'test').ls()

# Load Model

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.MODEL.WEIGHTS = "../input/sartorius-transfer-learning-train-test-model/output/model_0006049.pth" 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.40
cfg.TEST.DETECTIONS_PER_IMAGE = 1000
try:
    predictor = DefaultPredictor(cfg)
except:
    cfg.MODEL.DEVICE = "cpu"
    predictor = DefaultPredictor(cfg)

# Predicting

In [ ]:
with open('../input/sartorius-cell-instance-segmentation-coco/annotations_val.json', "r", encoding='utf-8') as f:
    images = json.load(f)['images']
    
train_df = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')
train_df = train_df.groupby('id').agg(list).reset_index()
for col in train_df.columns[2:]:
    train_df[col] = train_df[col].apply(
        lambda x: np.unique(x)[0] if len(np.unique(x)) == 1 else np.unique(x)
    )


pred_list_by_class = [[], [], []]
ture_masks_by_class = [[], [], []]
for image in images:
    fn = str(Dir_testdata) + '/' + image['file_name']
    im = cv2.imread(fn)
    pred = predictor(im)
    pred['image_path'] = fn
    pred_class = torch.mode(pred['instances'].pred_classes)[0]
    pred_list_by_class[pred_class].append(pred)

    fid = image['id']
    df = train_df[train_df['id']==fid]
    shape = df[['height', 'width']].values[0]
    rles = df['annotation'].values[0]
    masks = rles_to_mask(rles, shape).astype(np.uint16)
    ture_masks_by_class[pred_class].append(masks)

In [ ]:
THRESHOLD_RANGE = [x/100 for x in range(10, 91, 2)]
MIN_PIXELS_RANGE = range(30, 251, 5)
THRESHOLDS = [0.15, 0.2, 0.45]
MIN_PIXELS = [80, 160, 60]
#FIND_BEST_CHECKPOINT = False
FIND_THRESHOLDS = True

In [ ]:
#if FIND_BEST_CHECKPOINT:
#    do_eval()

In [ ]:
if FIND_THRESHOLDS:
    for cls_i in range(len(pred_list_by_class)):    
        best_iou = 0
        best_th = 0
        for th in THRESHOLD_RANGE:    
            preds = pred_list_by_class[cls_i]
            ture_masks = ture_masks_by_class[cls_i]
            ious = get_all_iou(preds, ture_masks, th, MIN_PIXELS[cls_i])
            iou = np.mean(ious)
            print(f'cls:{cls_i}, th: {th}, iou:{iou}')
            if best_iou < iou:
                best_iou = iou
                best_th = th
        THRESHOLDS[cls_i] = best_th

        best_iou = 0
        best_minp = 0
        for minp in MIN_PIXELS_RANGE:
            preds = pred_list_by_class[cls_i]
            ture_masks = ture_masks_by_class[cls_i]
            ious = get_all_iou(preds, ture_masks, THRESHOLDS[cls_i], minp)
            iou = np.mean(ious)
            print(f'cls:{cls_i}, minp: {minp}, iou:{iou}')
            if best_iou < iou:
                best_iou = iou
                best_minp = minp
        MIN_PIXELS[cls_i] = best_minp

In [ ]:
print(f'THRESHOLDS:" {THRESHOLDS}')
print(f'MIN_PIXELS:" {MIN_PIXELS}')

In [ ]:
#get bad preds
bad_pred_dict_list_list = []
for cls_i in range(len(pred_list_by_class)):  
    preds = pred_list_by_class[cls_i]
    ture_masks = ture_masks_by_class[cls_i]
    bad_pred_dict_list_list.append(get_bad_preds(preds, ture_masks, THRESHOLDS[cls_i], MIN_PIXELS[cls_i]))

# Visualize predictions

In [ ]:
cls_i = 0
bad_pred_dict_list = bad_pred_dict_list_list[cls_i]
for bad_pred_dict in bad_pred_dict_list:
    figure, axs = plt.subplots(1,3, figsize=(40,30))
    figure.suptitle(f"class: {cls_i}, score: {bad_pred_dict['score']}")
    axs[0].imshow(cv2.imread(bad_pred_dict['image_path']))
    axs[0].axis("off")
    axs[1].imshow(bad_pred_dict['pred_mask'])
    axs[1].axis("off")
    axs[2].imshow(bad_pred_dict['true_mask'])
    axs[2].axis("off")

In [ ]:
cls_i = 1
bad_pred_dict_list = bad_pred_dict_list_list[cls_i]
for bad_pred_dict in bad_pred_dict_list:
    figure, axs = plt.subplots(1,3, figsize=(40,30))
    figure.suptitle(f"class: {cls_i}, score: {bad_pred_dict['score']}")
    axs[0].imshow(cv2.imread(bad_pred_dict['image_path']))
    axs[0].axis("off")
    axs[1].imshow(bad_pred_dict['pred_mask'])
    axs[1].axis("off")
    axs[2].imshow(bad_pred_dict['true_mask'])
    axs[2].axis("off")

In [ ]:
cls_i = 2
bad_pred_dict_list = bad_pred_dict_list_list[cls_i]
for bad_pred_dict in bad_pred_dict_list:
    figure, axs = plt.subplots(1,3, figsize=(40,30))
    figure.suptitle(f"class: {cls_i}, score: {bad_pred_dict['score']}")
    axs[0].imshow(cv2.imread(bad_pred_dict['image_path']))
    axs[0].axis("off")
    axs[1].imshow(bad_pred_dict['pred_mask'])
    axs[1].axis("off")
    axs[2].imshow(bad_pred_dict['true_mask'])
    axs[2].axis("off")

# Submission

In [ ]:
import os

os.system(f'cp {cfg.MODEL.WEIGHTS} ./')
os.system(f'cp ../input/sartorius-transfer-learning-train-test-model/output/model_0008711.pth ./')
os.system(f'cp ../input/sartorius-transfer-learning-train-test-model/output/model_0008469.pth ./')

# References
* https://www.kaggle.com/slawekbiel/positive-score-with-detectron-3-3-inference